Zadanie będzie skończone do piątku 20.03 wieczorem

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk

In [4]:
d = {'x':[4,0],'y':[0,4]}
S = pd.DataFrame(d)    #macierz kowariancji
mt1 = np.array([-3,-1]) #średnie klas
mt2 = np.array([2,2])
n1 = 40                #liczebność klas
n2 = 30
n = n1+n2
X1 = np.random.multivariate_normal(mt1, S, n1) #losowanie obserwacji w klasach na podstawie wcześniejszych parametrów
X2 = np.random.multivariate_normal(mt2, S, n2)
g = 2